### Synthetic Matrics - random placement (uniform) - obj to see peak CSE gains
    - Vary Size
    - Vary number of distinct constants
    - Vary density

In [ ]:
import numpy as np
np.random.seed(42)
import random
random.seed(42)
import os

In [ ]:
class RandomMatrixValidateError(Exception):
    pass

In [ ]:
import os
class RandomMatrix():
    """
    Generate a random mxn sparse matrix.
    Guarantees use of min num_unique constants.
    """
    def __init__(self, dim_0, dim_1, density, num_unique):
        self.m = dim_0
        self.n = dim_1
        self.p = density
        self.uniq = num_unique

        self.name = "r_{}_c_{}_p_{}_uniq_{}".format(dim_0, dim_1, density, num_unique)

        self._validate_inputs()

        self.mat = self._fill_matrix()

    def _validate_inputs(self):
        size = self.m * self.n
        fill_amount = int(size * self.p)
        if fill_amount < self.uniq:
            raise RandomMatrixValidateError("Density too low for number of unique required")

    def _fill_matrix(self):
        draw_amount = int(self.m*self.n*self.p)
        # unique values to store in matrix
        a_constants = np.arange(start=1, stop=self.uniq+1, dtype='float64')
        
        mat = np.zeros(shape=[self.m, self.n])
        
        # build choices
        if draw_amount == self.p:
            choices = np.random.choice(a_constants, size=draw_amount, replace=False)
        else:
            choices = np.zeros(draw_amount)
            # use each constant at least once
            choices[:self.uniq] = np.random.choice(a_constants, size=self.uniq, replace=False)
            choices[self.uniq:] = np.random.choice(a_constants, size=draw_amount-self.uniq, replace=True)
            
        # fill mat with choices
        p = [(i,j) for i in range(self.m) for j in range(self.n)]
        positions = random.sample(p, draw_amount)
        
        for i in range(draw_amount):
            c = choices[i]
            _i, _j = positions[i]
            mat[_i, _j] = c

        return mat

    def save(self, dir):
        os.makedirs(dir, exist_ok=True)
        fname = os.path.join(dir, self.name+".txt")
        np.savetxt(fname, self.mat)        

### Init Config

In [ ]:
DEFAULT_DIM_0    = 128
DEFAULT_DIM_1    = 128
DEFAULT_DENSITY  = 0.05
DEFAULT_NUM_UNIQ = 16, 64, 256

PARENT_DIR = "./../synth_mats_wider"

## Vary rows

In [ ]:
rows = [32, 64, 128, 256, 512, 1024]

In [ ]:
n = DEFAULT_DIM_1
p = DEFAULT_DENSITY
for q in DEFAULT_NUM_UNIQ:
    DIR = os.path.join(PARENT_DIR, "vary_row/q_{}".format(q))
    for m in rows:
        try:
            mat = RandomMatrix(m,n,p,q)
            mat.save(DIR)
        except RandomMatrixValidateError:
            print("skipped {} {} {} {}".format(m,n,p,q))

## Vary cols

In [ ]:
cols = [32, 64, 128, 256, 512, 1024]

In [ ]:
m = DEFAULT_DIM_0
p = DEFAULT_DENSITY
for q in DEFAULT_NUM_UNIQ:
    DIR = os.path.join(PARENT_DIR, "vary_col/q_{}".format(q))
    for n in cols:
        try:
            mat = RandomMatrix(m,n,p,q)
            mat.save(DIR)
        except RandomMatrixValidateError:
            print("skipped {} {} {} {}".format(m,n,p,q))

## Vary density

In [ ]:
density = [0.01, 0.025, 0.05, 0.075, 0.1, 0.125, 0.15, 0.2, 0.25, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

In [ ]:
m = DEFAULT_DIM_0
n = DEFAULT_DIM_1
for q in DEFAULT_NUM_UNIQ:
    DIR = os.path.join(PARENT_DIR, "vary_density/q_{}".format(q))
    for p in density:
        try:
            mat = RandomMatrix(m,n,p,q)
            mat.save(DIR)
        except RandomMatrixValidateError:
            print("skipped {} {} {} {}".format(m,n,p,q))

## Vary num unique

In [ ]:
# unique = [8, 16, 24, 31, 40, 48, 56, 64, 78, 96, 112, 128, 144, 160, 176, 192, 208, 224, 240, 256]
unique = np.arange(16, 256+1, 16)
print(unique)

In [ ]:
m = DEFAULT_DIM_0
n = DEFAULT_DIM_1
p = DEFAULT_DENSITY
DIR = os.path.join(PARENT_DIR, "vary_unique/")
for q in unique:
    try:
        mat = RandomMatrix(m,n,p,q)
        mat.save(DIR)
    except RandomMatrixValidateError:
        print("skipped {} {} {} {}".format(m,n,p,q))